# More Advanced Thermal Emission

In this example, we'll look at the emission from a disk galaxy from the Illustris TNG simulations.
This dataset has metallicity information for several species in it. We'll make a cut in phase space like we did in the previous example. The dataset we want to use for this example is available for download [here](https://hea-www.cfa.harvard.edu/~jzuhone/cutout_37.hdf5). 

First, import our necessary modules:

In [ ]:
import yt
import pyxsim
import soxs

We will make phase space cuts on the gas cells using density, temperature, and star formation rate:

In [ ]:
# Note that the units of all numbers in this function are CGS
# define hot gas filter 
def hot_gas(pfilter, data):
    pfilter1 = data[pfilter.filtered_type, "temperature"] > 3.0e5
    pfilter2 = data["PartType0", "StarFormationRate"] == 0.0
    pfilter3 = data[pfilter.filtered_type, "density"] < 5e-25
    return pfilter1 & pfilter2 & pfilter3
yt.add_particle_filter("hot_gas", function=hot_gas,
                       filtered_type='gas', requires=["temperature","density"])

Next, we `load` the dataset with yt, and add the `"hot_gas"` filter to the dataset: 

In [ ]:
ds = yt.load('cutout_37.hdf5', 
             bounding_box=[[-1000.0, 1000], [-1000.0, 1000], [-1000.0, 1000]])
ds.add_particle_filter("hot_gas")

We also need to tell pyXSIM which elements have fields in the dataset that 
should be used. To do this we create a `var_elem` dictionary of (key, value) 
pairs corresponding to the element name and the yt field name (assuming the 
`"hot_gas"` type).

In [ ]:
# metal fields to use
metals = ["C_fraction", "N_fraction", "O_fraction",
          "Ne_fraction", "Mg_fraction", "Si_fraction", "Fe_fraction"]
var_elem = {elem.split("_")[0]: ("hot_gas", elem) for elem in metals}


Now that we have everything we need, we'll set up the `IGMSourceModel`, which is based on Cloudy and includes resonant scattering off of the CXB (see [here](https://hea-www.cfa.harvard.edu/~jzuhone/pyxsim/source_models/thermal_sources.html#igm-source-model) for more details). Because we created a hot gas filter, we will use the `"hot_gas"` field type for the emission measure, temperature, and metallicity fields. 

In [ ]:
source_model = pyxsim.IGMSourceModel(
    0.1, 4.0, 5000, ("hot_gas","metallicity"),
    binscale="log", resonant_scattering=True,
    temperature_field=("hot_gas","temperature"),
    emission_measure_field=("hot_gas", "emission_measure"),
    nh_field=("hot_gas", "H_nuclei_density"),
    var_elem=var_elem
)

As in other examples, we choose big numbers for the collecting area and exposure time, and a redshift:

In [ ]:
exp_time = (1.0, "Ms") # exposure time
area = (5000.0, "cm**2") # collecting area
redshift = 0.01

Next, we'll create a box object to serve as a source for the photons. The dataset consists of only
the galaxy at a specific location, which we use below, and pick a width of 1 Mpc:

In [ ]:
c = ds.arr([0.0, 0.0, 0.0], "code_length")
width = ds.quan(1.0, "Mpc")
le = c - 0.5*width
re = c + 0.5*width
box = ds.box(le, re)

So, that's everything--let's create the photons! We use the `make_photons` function for this:

In [ ]:
n_photons, n_cells = pyxsim.make_photons("cutout_37_photons", box, redshift, area, 
                                         exp_time, source_model)

And now we create events using the `project_photons` function. Let's project along the `"z"` axis. We'll use the `"tbabs"` foreground absorption model this time, with a neutral hydrogen column of $N_H = 2 \times 10^{20}~{\rm cm}^{-2}$:

In [ ]:
n_events = pyxsim.project_photons("cutout_37_photons", "cutout_37_events", "x", (30.,45.), 
                                  absorb_model="tbabs", nH=0.02)

Now that we have a set of "events" on the sky, we can read them in and write them to a SIMPUT file:

In [ ]:
events = pyxsim.EventList("cutout_37_events.h5")
events.write_to_simput("cutout_37", overwrite=True)

We can then use this SIMPUT file as an input to the instrument simulator in SOXS. We'll observe it with the 2eV LEM model for 1 Ms. First, we'll create a background file that we'll use for the background:

In [ ]:
soxs.make_background_file("bkgnd_evt_37.fits", (1000.0, "ks"), "lem_2eV",
                          [30.0, 45.0], overwrite=True)

Now we simulate the source itself, adding in the background:

In [ ]:
soxs.instrument_simulator("cutout_37_simput.fits", "evt_37.fits", (1000.0, "ks"), 
                          "lem_2eV", [30., 45.], overwrite=True, 
                          bkgnd_file="bkgnd_evt_37.fits")

We can use the `write_image()` function in SOXS to bin the events into an image and write them 
to a file, restricting the energies between 0.644 and 0.65 keV, which focuses on the redshifted OVIII line:

In [ ]:
soxs.write_image("evt_37.fits", "img_37.fits", emin=0.644, emax=0.65, overwrite=True)

Now we can take a quick look at the image: 

In [ ]:
soxs.plot_image("img_37.fits", stretch='log', cmap='arbre', width=0.4, vmin=0.5)

Now we will make spectra to look at. First, filter the events of both the combined source and background files and the background-only files within 0.15 degree of the center:

In [ ]:
soxs.filter_events("evt_37.fits", "evt_37_filter.fits", overwrite=True,
                   region='fk5\ncircle(30.0000000,45.0000000,540.000")')
soxs.filter_events("bkgnd_evt_37.fits", "bkgnd_evt_37_filter.fits", overwrite=True,
                   region='fk5\ncircle(30.0000000,45.0000000,540.000")')


Now bin up spectra for these new event files:

In [ ]:
soxs.write_spectrum("evt_37_filter.fits", "evt_37.pi", overwrite=True)
soxs.write_spectrum("bkgnd_evt_37_filter.fits", "bkgnd_evt_37.pi", overwrite=True)

Finally, we can plot the spectra. Below, the total spectrum is in blue and the background/foreground spectrum is in orange. The lines from the emission of the distant galaxy are redshifted away from the foreground Milky Way lines.

In [ ]:
fig, ax = soxs.plot_spectrum("evt_37.pi", xmin=0.5, xmax=0.7, xscale="linear", ymin=0.5)
soxs.plot_spectrum("bkgnd_evt_37.pi", xmin=0.5, xmax=0.7, fig=fig, ax=ax, ymin=0.5)